In [1]:
import os
os.chdir('..')

### 1. Read data

In [31]:
os.listdir('data/input')

['cybersecurity-reports', 'muc4']

In [67]:
import json
from collections import OrderedDict

path = 'data/input/cybersecurity-reports/train_full.json'
with open(path, "r") as doc_keys_file:
    input_roles = json.load(doc_keys_file, object_pairs_hook=OrderedDict)
    
input_roles['fireeye-1']#['doc']

OrderedDict([('doc',
              'Chinese cyber espionage operators modified the software packages of a legitimate vendor, NetSarang Computer, allowing access to a broad range of industries and institutions that include financial services, transportation, telecommunications, energy, media, academic, retail, and gaming. The malicious actors embedded a backdoor named SHADOWPAD, which can access network data and download additional malware. Based on shared communications infrastructure and similar targeting and tactics, FireEye Threat Intelligence assesses this activity is related to a Chinese espionage team that previously targeted a gaming company in 2015.'),
             ('roles',
              OrderedDict([('adversary',
                            [['Chinese cyber espionage operators',
                              'malicious actors',
                              'Chinese espionage team']]),
                           ('capability', [['backdoor', 'SHADOWPAD']]),
                   

In [56]:
input_roles.copy()

OrderedDict([('fireeye-1',
              OrderedDict([('doc',
                            'Chinese cyber espionage operators modified the software packages of a legitimate vendor, NetSarang Computer, allowing access to a broad range of industries and institutions that include financial services, transportation, telecommunications, energy, media, academic, retail, and gaming. The malicious actors embedded a backdoor named SHADOWPAD, which can access network data and download additional malware. Based on shared communications infrastructure and similar targeting and tactics, FireEye Threat Intelligence assesses this activity is related to a Chinese espionage team that previously targeted a gaming company in 2015.'),
                           ('roles',
                            OrderedDict([('adversary',
                                          [['Chinese cyber espionage operators',
                                            'malicious actors',
                                       

### 2. Load tokenizer and preprocessor

In [44]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', use_fast=False)

# this separates the paragraphs into sentences
import spacy
nlp = spacy.load("en_core_web_sm")

In [8]:
import random
from copy import deepcopy
import spacy
from transformers import AutoTokenizer

class Doc_Seq:
    def __init__(self, key, seq_tag_pair, doc):
        self.key = key
        self.seq_tag_pair = seq_tag_pair
        self.doc = doc
    
    def __repr__(self):
        return f'Doc_Seq(key={self.key}, doc={self.doc.text})'

def create_sent_tagging(input_roles, test_set=False, bert_model = 'bert-base-uncased', keys_dict_none_empty=None):
    """
    :param input_roles: ordered dict containing key, doc and roles
    :param test_set: boolean to indicate whether we are processing train or test set
    :param keys_dict_none_empty: list of keys to ignore
    """

    tokenizer = AutoTokenizer.from_pretrained(bert_model, use_fast=False)
    nlp = spacy.load("en_core_web_sm")
    
    doc_keys = deepcopy(input_roles)
    
    # turn doc_keys (entity) into doc_keys (mentions)
    for docid in doc_keys:
        for role in doc_keys[docid]["roles"]:
            mentions = list()
            for entity in doc_keys[docid]["roles"][role]:
                for mention in entity:
                    if mention not in mentions:
                        mentions.append(mention)
            doc_keys[docid]["roles"][role] = mentions

    seqs_all_o = []
    seqs_not_all_o = []
    all_examples = []
    para_lens = []
    # summ = 0
    for key in doc_keys:
        
        # keys to ignore
        if keys_dict_none_empty is not None:
            if key not in keys_dict_none_empty: continue

        # get and sort doc-level spans to extract from doc key
        doc = doc_keys[key]["doc"]
        tags_values = doc_keys[key]["roles"]
        for tag in tags_values:
            values = tags_values[tag]

            values.sort(key=lambda x: len(x) *(-1)) # longest first
            # values.sort(key=lambda x: len(x))     # shortest first

        # sample tag_values
        # OrderedDict([('perp_individual_id', ['TERRORISTS', 'TERRORIST']), 
        # ('perp_organization_id', ['FARABUNDO MARTI NATIONAL LIBERATION FRONT', 'MARTI NATIONAL', 'FMLN']),
        # ('phys_tgt_id', ['LAS CANAS BRIDGE']), ('hum_tgt_name', []), 
        # ('incident_instrument_id', ['MORTAR', 'RIFLE'])])

        # get all the sentences from this doc key
        # JT: Split the documents by paragraphs
        paragraphs = doc.split("\n\n")
        doc_sents = []
        for para in paragraphs:
            para2 = " ".join(para.split("\n"))#.lower()
            para2 = nlp(para2)
            cnt = 0
            for sent in para2.sents:
                cnt += 1
                doc_sents.append(sent.text)
            para_lens.append(cnt)


        # get seqs and annotate
        # JT: Split long paragraphs by sentences
        num_sent_to_include = 3
        for idx in range(len(doc_sents)):
            if test_set:
                start = idx * num_sent_to_include
            else:
                start = idx
            end = start + num_sent_to_include
            if start >= len(doc_sents): break

            if end > len(doc_sents): end = len(doc_sents)

            # JT: sequence is a long string of the paragraph
            sequence = " ".join(doc_sents[start: end])
            
            all_o = True
            seq_tag_pair = []
            spacy_doc = nlp(sequence.lower())
            for tok in spacy_doc:
                # JT: my own modification to keep track of spacy tokens
                subword_tokens = tokenizer.tokenize(tok.text)
                for subword_token in subword_tokens:
                    seq_tag_pair.append([subword_token, 'O', tok.i])
            # seq_tokenized = tokenizer.tokenize(sequence)
            # seq_tag_pair = [[token, 'O'] for token in seq_tokenized]
            
            # JT: this iterates through the roles
            for tag_anno in tags_values:
                values = tags_values[tag_anno]
                for value in values:
                    value_tokenized = tokenizer.tokenize(value)
                    for idx, token_tag in enumerate(seq_tag_pair):
                        token, tag, word_idx = token_tag[0], token_tag[1], token_tag[2]
                        if token == value_tokenized[0]:
                            start, end = idx, idx + len(value_tokenized)
                            if end <= len(seq_tag_pair):
                                candidate = [x[0] for x in seq_tag_pair[start: end]]
                                tags = [x[1] for x in seq_tag_pair[start: end]] 

                                already_annoted = False
                                for tag in tags: 
                                    if tag != 'O': already_annoted = True
                                if already_annoted: continue

                                if " ".join(candidate) == " ".join(value_tokenized):
                                    all_o = False
                                    seq_tag_pair[start][1] = "B-" + tag_anno
                                    for i in range(start + 1, end):
                                        seq_tag_pair[i][1] = "I-" + tag_anno

            example = {'doc_id':key, 'seq': seq_tag_pair, 'text':sequence}
            # example = doc(key, seq_tag_pair, spacy_doc)
            all_examples.append(example)
            if not all_o:
                seqs_not_all_o.append(example)
            else:
                seqs_all_o.append(example)
            print(seq_tag_pair)

    # print(seqs_all_o, len(seqs_not_all_o))
    seqs_all_o_sample = random.sample(seqs_all_o, min(len(seqs_not_all_o), len(seqs_all_o)) )
    all_examples_sample_neg = seqs_not_all_o + seqs_all_o_sample
    print("Average paragraph sent # :", sum(para_lens)/len(para_lens))

    return all_examples_sample_neg, all_examples
    
all_examples_sample_neg, all_examples = create_sent_tagging(input_roles)

[['chinese', 'B-adversary', 0], ['cyber', 'I-adversary', 1], ['espionage', 'I-adversary', 2], ['operators', 'I-adversary', 3], ['modified', 'O', 4], ['the', 'O', 5], ['software', 'O', 6], ['packages', 'O', 7], ['of', 'O', 8], ['a', 'O', 9], ['legitimate', 'B-victim', 10], ['vendor', 'I-victim', 11], [',', 'I-victim', 12], ['nets', 'B-victim', 13], ['##aran', 'I-victim', 13], ['##g', 'I-victim', 13], ['computer', 'I-victim', 14], [',', 'O', 15], ['allowing', 'O', 16], ['access', 'O', 17], ['to', 'O', 18], ['a', 'O', 19], ['broad', 'O', 20], ['range', 'O', 21], ['of', 'O', 22], ['industries', 'O', 23], ['and', 'O', 24], ['institutions', 'O', 25], ['that', 'O', 26], ['include', 'O', 27], ['financial', 'B-victim', 28], ['services', 'I-victim', 29], [',', 'O', 30], ['transportation', 'O', 31], [',', 'O', 32], ['telecommunications', 'O', 33], [',', 'O', 34], ['energy', 'O', 35], [',', 'O', 36], ['media', 'O', 37], [',', 'O', 38], ['academic', 'O', 39], [',', 'O', 40], ['retail', 'O', 41], ['

[['the', 'O', 0], ['manner', 'O', 1], ['in', 'O', 2], ['which', 'O', 3], ['this', 'B-capability', 4], ['system', 'I-capability', 5], ['operates', 'O', 6], ['suggests', 'O', 7], ['that', 'O', 8], ['each', 'O', 9], ['gt', 'O', 10], ['##ag', 'O', 10], ['value', 'O', 11], ['is', 'O', 12], ['associated', 'O', 13], ['with', 'O', 14], ['a', 'O', 15], ['different', 'O', 16], ['trick', 'B-adversary', 17], ['##bot', 'I-adversary', 17], ['customer', 'I-adversary', 18], ['.', 'O', 19], ['fire', 'O', 20], ['##eye', 'O', 20], ['threat', 'O', 21], ['intelligence', 'O', 22], ['has', 'O', 23], ['identified', 'O', 24], ['and', 'O', 25], ['clustered', 'O', 26], ['a', 'O', 27], ['number', 'O', 28], ['of', 'O', 29], ['trick', 'B-capability', 30], ['##bot', 'I-capability', 30], ['gt', 'O', 31], ['##ag', 'O', 31], ['values', 'O', 32], ['found', 'O', 33], ['in', 'O', 34], ['the', 'O', 35], ['configuration', 'O', 36], ['of', 'O', 37], ['this', 'B-capability', 38], ['system', 'I-capability', 39], ['and', 'O', 4

[['it', 'O', 0], ['consists', 'O', 1], ['of', 'O', 2], ['high', 'O', 3], ['-', 'O', 4], ['level', 'O', 5], ['analysis', 'O', 6], ['and', 'O', 7], ['is', 'O', 8], ['not', 'O', 9], ['meant', 'O', 10], ['to', 'O', 11], ['provide', 'O', 12], ['in', 'O', 13], ['-', 'O', 14], ['depth', 'O', 15], ['insights', 'O', 16], ['into', 'O', 17], ['the', 'O', 18], ['nu', 'O', 19], ['##ances', 'O', 19], ['of', 'O', 20], ['each', 'O', 21], ['incident', 'O', 22], ['.', 'O', 23], ['our', 'O', 24], ['methodology', 'O', 25], ['evaluate', 'O', 26], ['##s', 'O', 26], ['four', 'O', 27], ['categories', 'O', 28], [',', 'O', 29], ['which', 'O', 30], ['are', 'O', 31], ['target', 'O', 32], [',', 'O', 33], ['so', 'O', 34], ['##phi', 'O', 34], ['##stic', 'O', 34], ['##ation', 'O', 34], [',', 'O', 35], ['impact', 'O', 36], [',', 'O', 37], ['and', 'O', 38], ['affected', 'O', 39], ['equipment', 'O', 40], ['architecture', 'O', 41], ['based', 'O', 42], ['on', 'O', 43], ['the', 'O', 44], ['purdue', 'O', 45], ['model', 'O',

[['fire', 'O', 0], ['##eye', 'O', 0], ['threat', 'O', 1], ['intelligence', 'O', 2], ['also', 'O', 3], ['observed', 'O', 4], ['changes', 'O', 5], ['in', 'O', 6], ['the', 'O', 7], ['way', 'O', 8], ['em', 'B-capability', 9], ['##ote', 'I-capability', 9], ['##t', 'I-capability', 9], ['communicate', 'O', 10], ['##s', 'O', 10], ['through', 'O', 11], ['http', 'B-infrastructure', 12], ['post', 'I-infrastructure', 13], ['requests', 'I-infrastructure', 14], ['with', 'O', 15], ['its', 'O', 16], ['command', 'B-capability', 17], ['and', 'I-capability', 18], ['control', 'I-capability', 19], ['(', 'I-capability', 20], ['c', 'I-capability', 21], ['&', 'I-capability', 21], ['amp', 'I-capability', 21], [';', 'I-capability', 21], ['c', 'I-capability', 21], [')', 'I-capability', 22], ['servers', 'I-capability', 23], ['.', 'O', 24], ['these', 'O', 25], ['changes', 'O', 26], ['were', 'O', 27], ['likely', 'O', 28], ['made', 'O', 29], ['to', 'O', 30], ['evade', 'O', 31], ['host', 'O', 32], ['and', 'O', 33], [

[['the', 'O', 0], ['ina', 'O', 1], ['##ct', 'O', 1], ['##ivity', 'O', 1], ['of', 'O', 2], ['em', 'B-adversary', 3], ['##ote', 'I-adversary', 3], ['##t', 'I-adversary', 3], ['distribution', 'I-adversary', 4], ['operations', 'I-adversary', 5], ['is', 'O', 6], ['supported', 'O', 7], ['by', 'O', 8], ['evidence', 'O', 9], ['observed', 'O', 10], ['across', 'O', 11], ['the', 'O', 12], ['fire', 'O', 13], ['##eye', 'O', 13], ['sensor', 'O', 14], ['network', 'O', 15], ['and', 'O', 16], ['active', 'O', 17], ['monitoring', 'O', 18], ['of', 'O', 19], ['the', 'O', 20], ['em', 'B-capability', 21], ['##ote', 'I-capability', 21], ['##t', 'I-capability', 21], ['bot', 'I-capability', 22], ['##net', 'I-capability', 22], ['.', 'O', 23], ['the', 'O', 24], ['cause', 'O', 25], ['for', 'O', 26], ['this', 'O', 27], ['current', 'O', 28], ['ce', 'O', 29], ['##ssa', 'O', 29], ['##tion', 'O', 29], ['of', 'O', 30], ['activity', 'O', 31], ['is', 'O', 32], ['unclear', 'O', 33], [',', 'O', 34], ['and', 'O', 35], ['it',

...

[['out', 'O', 1], ['of', 'O', 2], ['the', 'O', 3], ['three', 'O', 4], ['flaws', 'O', 5], ['impact', 'O', 6], ['##ing', 'O', 6], ['the', 'O', 7], ['cisco', 'O', 8], ['small', 'O', 9], ['business', 'O', 10], ['220', 'O', 11], ['series', 'O', 12], ['smart', 'O', 13], ['switches', 'O', 14], [',', 'O', 15], ['two', 'O', 16], ['are', 'O', 17], ['critical', 'B-capability', 18], ['remote', 'I-capability', 19], ['code', 'I-capability', 20], ['execution', 'I-capability', 21], ['(', 'O', 22], ['cv', 'O', 23], ['##e', 'O', 23], ['-', 'O', 23], ['2019', 'O', 23], ['-', 'O', 24], ['1913', 'O', 25], [')', 'O', 26], ['and', 'O', 27], ['authentication', 'B-capability', 28], ['bypass', 'I-capability', 29], ['security', 'I-capability', 30], ['(', 'O', 31], ['cv', 'O', 32], ['##e', 'O', 32], ['-', 'O', 32], ['2019', 'O', 32], ['-', 'O', 33], ['1912', 'O', 34], [')', 'O', 35], ['issues', 'O', 36], ['.', 'O', 37], ['an', 'B-adversary', 39], ['una', 'I-adversary', 40], ['##uth', 'I-adversary', 40], ['##ent',

[['as', 'O', 0], ['a', 'O', 1], ['result', 'O', 2], [',', 'O', 3], ['matt', 'O', 5], ['decided', 'O', 6], ['to', 'O', 7], ['show', 'O', 8], ['how', 'O', 9], ['the', 'O', 10], ['vulnerability', 'O', 11], ['could', 'O', 12], ['be', 'O', 13], ['exploited', 'O', 14], ['and', 'O', 15], ['published', 'O', 16], ['a', 'O', 17], ['po', 'O', 18], ['##c', 'O', 18], ['where', 'O', 20], ['he', 'O', 21], ['gets', 'O', 22], ['a', 'O', 23], ['command', 'O', 24], ['prompt', 'O', 25], ['window', 'O', 26], ['running', 'O', 27], ['with', 'O', 28], ['system', 'O', 30], ['account', 'O', 31], ['privileges', 'O', 32], [',', 'O', 33], ['the', 'O', 34], ['highest', 'O', 35], ['for', 'O', 36], ['a', 'O', 37], ['user', 'O', 38], ['on', 'O', 39], ['windows', 'O', 40], ['.', 'O', 41], ['fast', 'O', 43], ['forward', 'O', 44], ['a', 'O', 45], ['few', 'O', 46], ['days', 'O', 47], ['and', 'O', 48], ['valve', 'B-victim', 49], ['published', 'O', 50], ['a', 'O', 51], ['patch', 'O', 52], ['for', 'O', 54], ['the', 'O', 55],

[['this', 'O', 0], ['comes', 'O', 1], ['after', 'O', 2], ['being', 'O', 3], ['in', 'O', 4], ['##ert', 'O', 4], ['since', 'O', 5], ['the', 'O', 6], ['beginning', 'O', 7], ['of', 'O', 8], ['june', 'O', 9], ['.', 'O', 10], ['although', 'O', 12], ['it', 'O', 13], ['started', 'O', 14], ['as', 'O', 15], ['a', 'O', 16], ['banking', 'B-capability', 17], ['trojan', 'I-capability', 18], ['in', 'O', 19], ['2014', 'O', 20], [',', 'O', 21], ['em', 'B-capability', 22], ['##ote', 'I-capability', 22], ['##t', 'I-capability', 22], ['changed', 'O', 23], ['its', 'O', 24], ['course', 'O', 25], ['to', 'O', 26], ['becoming', 'O', 27], ['a', 'O', 28], ['bot', 'O', 29], ['##net', 'O', 29], ['that', 'O', 30], ['delivers', 'O', 31], ['various', 'O', 32], ['mal', 'B-capability', 33], ['##ware', 'I-capability', 33], ['strains', 'O', 34], ['.', 'O', 35]]
[['although', 'O', 1], ['it', 'O', 2], ['started', 'O', 3], ['as', 'O', 4], ['a', 'O', 5], ['banking', 'B-capability', 6], ['trojan', 'I-capability', 7], ['in', '

[['"', 'O', 1], ['apart', 'O', 2], ['from', 'O', 3], ['a', 'O', 4], ['few', 'O', 5], ['pun', 'O', 6], ['##ct', 'O', 6], ['##uation', 'O', 6], ['errors', 'O', 7], ['and', 'O', 8], ['the', 'O', 9], ['missing', 'O', 10], ['space', 'O', 11], ['before', 'O', 12], ['the', 'O', 13], ['word', 'O', 14], ['‘', 'O', 15], ['please', 'O', 16], ['’', 'O', 17], [',', 'O', 18], ['this', 'O', 19], ['message', 'O', 20], ['is', 'O', 21], ['clean', 'O', 22], [',', 'O', 23], ['clear', 'O', 24], ['and', 'O', 25], ['low', 'O', 26], ['-', 'O', 27], ['key', 'O', 28], ['enough', 'O', 29], ['not', 'O', 30], ['to', 'O', 31], ['raise', 'O', 32], ['instant', 'O', 33], ['alarm', 'O', 34], ['bells', 'O', 35], [',', 'O', 36], ['"', 'O', 37], ['details', 'O', 38], ['so', 'O', 39], ['##ph', 'O', 39], ['##os', 'O', 39], ["'", 'O', 40], ['paul', 'O', 42], ['duck', 'O', 43], ['##lin', 'O', 43], ['who', 'O', 45], ['analyzed', 'O', 46], ['the', 'O', 47], ['campaign', 'O', 48], ['.', 'O', 49]]
[['"', 'O', 0], ['apart', 'O', 1

[['besides', 'O', 1], ['the', 'O', 2], ['blue', 'O', 3], ['##tooth', 'O', 3], ['speakers', 'O', 4], ['no', 'O', 5], ['longer', 'O', 6], ['working', 'O', 7], ['properly', 'O', 8], [',', 'O', 9], ['users', 'O', 10], ['will', 'O', 11], ['also', 'O', 12], ['notice', 'O', 13], ['a', 'O', 14], ['microsoft', 'O', 16], ['blue', 'O', 17], ['##tooth', 'O', 17], ['a2', 'O', 18], ['##dp', 'O', 18], ['source', 'O', 19], ['entry', 'O', 20], ['under', 'O', 21], ['the', 'O', 22], ['sound', 'O', 24], [',', 'O', 25], ['video', 'O', 26], ['and', 'O', 27], ['game', 'O', 28], ['controllers', 'O', 29], ['node', 'O', 30], ['displaying', 'O', 31], ['a', 'O', 32], ['yellow', 'O', 33], ['ex', 'O', 34], ['##cl', 'O', 34], ['##ama', 'O', 34], ['##tion', 'O', 34], ['mark', 'O', 35], ['icon', 'O', 36], ['in', 'O', 37], ['device', 'O', 38], ['manager', 'O', 39], ['.', 'O', 40]]
[['besides', 'O', 0], ['the', 'O', 1], ['blue', 'O', 2], ['##tooth', 'O', 2], ['speakers', 'O', 3], ['no', 'O', 4], ['longer', 'O', 5], ['wo

In [9]:
all_examples[:3]

[{'doc_id': 'fireeye-1',
  'seq': [['chinese', 'B-adversary', 0],
   ['cyber', 'I-adversary', 1],
   ['espionage', 'I-adversary', 2],
   ['operators', 'I-adversary', 3],
   ['modified', 'O', 4],
   ['the', 'O', 5],
   ['software', 'O', 6],
   ['packages', 'O', 7],
   ['of', 'O', 8],
   ['a', 'O', 9],
   ['legitimate', 'B-victim', 10],
   ['vendor', 'I-victim', 11],
   [',', 'I-victim', 12],
   ['nets', 'B-victim', 13],
   ['##aran', 'I-victim', 13],
   ['##g', 'I-victim', 13],
   ['computer', 'I-victim', 14],
   [',', 'O', 15],
   ['allowing', 'O', 16],
   ['access', 'O', 17],
   ['to', 'O', 18],
   ['a', 'O', 19],
   ['broad', 'O', 20],
   ['range', 'O', 21],
   ['of', 'O', 22],
   ['industries', 'O', 23],
   ['and', 'O', 24],
   ['institutions', 'O', 25],
   ['that', 'O', 26],
   ['include', 'O', 27],
   ['financial', 'B-victim', 28],
   ['services', 'I-victim', 29],
   [',', 'O', 30],
   ['transportation', 'O', 31],
   [',', 'O', 32],
   ['telecommunications', 'O', 33],
   [',', 'O'

### Test function

In [1]:
import os
os.chdir('..')

import json
from collections import OrderedDict

path = 'data/input/cybersecurity-reports/train_full.json'
with open(path, "r") as doc_keys_file:
    input_roles = json.load(doc_keys_file, object_pairs_hook=OrderedDict)
    
input_roles['fireeye-1']#['doc']

from src.preprocessing import create_sent_tagging
all_examples_sample_neg, all_examples = create_sent_tagging(input_roles)
all_examples_sample_neg[:3]

Average paragraph sent # : 3.515695067264574


[{'doc_id': 'fireeye-1',
  'seq': [['chinese', 'B-adversary', 0],
   ['cyber', 'I-adversary', 1],
   ['espionage', 'I-adversary', 2],
   ['operators', 'I-adversary', 3],
   ['modified', 'O', 4],
   ['the', 'O', 5],
   ['software', 'O', 6],
   ['packages', 'O', 7],
   ['of', 'O', 8],
   ['a', 'O', 9],
   ['legitimate', 'B-victim', 10],
   ['vendor', 'I-victim', 11],
   [',', 'I-victim', 12],
   ['nets', 'B-victim', 13],
   ['##aran', 'I-victim', 13],
   ['##g', 'I-victim', 13],
   ['computer', 'I-victim', 14],
   [',', 'O', 15],
   ['allowing', 'O', 16],
   ['access', 'O', 17],
   ['to', 'O', 18],
   ['a', 'O', 19],
   ['broad', 'O', 20],
   ['range', 'O', 21],
   ['of', 'O', 22],
   ['industries', 'O', 23],
   ['and', 'O', 24],
   ['institutions', 'O', 25],
   ['that', 'O', 26],
   ['include', 'O', 27],
   ['financial', 'B-victim', 28],
   ['services', 'I-victim', 29],
   [',', 'O', 30],
   ['transportation', 'O', 31],
   [',', 'O', 32],
   ['telecommunications', 'O', 33],
   [',', 'O'

### Preprocess all files

In [2]:
path = "data/input/cybersecurity-reports/"
filenames = ['dev_full.json', 'test.json', 'train_full.json']

In [3]:
for filename in filenames:
    with open(path + filename, "r") as doc_keys_file:
        input_roles = json.load(doc_keys_file, object_pairs_hook=OrderedDict)

    from src.preprocessing import create_sent_tagging
    all_examples_sample_neg, all_examples = create_sent_tagging(input_roles)
    
    # writing to jsonlines file
    with open('data/processed/' + filename+'l', 'a', encoding='utf-8') as f:
        for line in all_examples_sample_neg:
            f.write(json.dumps(line, ensure_ascii=False) + '\n')

Average paragraph sent # : 3.6363636363636362
Average paragraph sent # : 3.090909090909091
Average paragraph sent # : 3.515695067264574
